In [1]:
with open('cleaned-news.txt') as fopen:
    data = fopen.read().split('\n')
    
len(data)

3656919

In [2]:
data[:5]

['Timb menteri pertanian',
 'China telah menjadi pasaran antarabangsa baru Malaysia untuk pelbagai nanas premium, MD2, berikutan kejayaan durian dalam menembusi negara itu pada Mei lepas.',
 'Timbalan Menteri Pertanian dan Industri Asas Tani Sim Tze Tzin kementerian yakin dapat mencapai eksport keseluruhan nanas sebanyak RM350 juta tahun ini berbanding RM343 juta yang dicatatkan pada tahun lepas dengan pasaran baharu itu.',
 '"Industri nanas negara dijangka mempunyai kadar pertumbuhan tahunan sebanyak lima peratus setahun berdasarkan trend yang semakin meningkat dan penerimaan terhadap nanas Malaysia.',
 '"Pengeluaran nanas kebangsaan pada 2019 dijangka bernilai RM530 juta berbanding RM502 juta pada 2018," katanya kepada media selepas pelancaran penerbangan sewa khas Malaysia yang pertama untuk membawa nanas dari Malaysia ke Zhengzhou, China, di Sepang hari ini.']

In [3]:
results, result = [], []
for i in data:
    if not len(i) and len(result):
        results.append(result)
        result = []
    else:
        result.append(i)
        
if len(result):
    results.append(result)
    
print(len(results))

173012


In [4]:
import re

def sliding(strings, n = 5):
    results = []
    for i in range(len(strings) - n):
        results.append(strings[i: i + n])
    return results

def _get_ngrams(n, text):
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set


def _get_word_ngrams(n, sentences):
    assert len(sentences) > 0
    assert n > 0

    words = sum(sentences, [])
    return _get_ngrams(n, words)

def cal_rouge(evaluated_ngrams, reference_ngrams):
    reference_count = len(reference_ngrams)
    evaluated_count = len(evaluated_ngrams)

    overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
    overlapping_count = len(overlapping_ngrams)

    if evaluated_count == 0:
        precision = 0.0
    else:
        precision = overlapping_count / evaluated_count

    if reference_count == 0:
        recall = 0.0
    else:
        recall = overlapping_count / reference_count

    f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))
    return {"f": f1_score, "p": precision, "r": recall}

def _rouge_clean(s):
    return re.sub(r'[^a-zA-Z0-9 ]', '', s)

In [5]:
import numpy as np
from numpy.random import default_rng

rng = default_rng()

In [9]:
def get_rouges(strings, n = 1):
    rouges = []
    for i in range(len(strings)):
        abstract = strings[i]
        doc_sent_list = [strings[k] for k in range(len(strings)) if k != i]
        sents = _rouge_clean(' '.join(doc_sent_list)).split()
        abstract = _rouge_clean(abstract).split()
        evaluated_1grams = _get_word_ngrams(n, [sents])
        reference_1grams = _get_word_ngrams(n, [abstract])
        rouges.append(cal_rouge(evaluated_1grams, reference_1grams)['f'])
    return rouges

# Principal Select top-m scored sentences according to importance. As a proxy for importance we compute ROUGE1-
# F1 (Lin, 2004) between the sentence and the rest of the document
def get_rouge(strings, top_k = 1):
    rouges = get_rouges(strings)
    return np.argsort(rouges)[::-1][:top_k]

# Random Uniformly select m sentences at random.
def get_random(strings, top_k = 1):
    return rng.choice(len(strings), size=top_k, replace=False)

# Lead Select the first m sentences.
def get_lead(strings, top_k = 1):
    return [i for i in range(top_k)]

[0]

In [8]:
slided = sliding(results[0][:])
for i in range(len(slided)):
    strings = slided[i]
    rouge_, random_ = get_rouge(strings), get_random(strings)
    strings[rouge_[0]] = '[MASK]'
    print(strings, '\n')

['Timb menteri pertanian', 'China telah menjadi pasaran antarabangsa baru Malaysia untuk pelbagai nanas premium, MD2, berikutan kejayaan durian dalam menembusi negara itu pada Mei lepas.', '[MASK]', '"Industri nanas negara dijangka mempunyai kadar pertumbuhan tahunan sebanyak lima peratus setahun berdasarkan trend yang semakin meningkat dan penerimaan terhadap nanas Malaysia.', '"Pengeluaran nanas kebangsaan pada 2019 dijangka bernilai RM530 juta berbanding RM502 juta pada 2018," katanya kepada media selepas pelancaran penerbangan sewa khas Malaysia yang pertama untuk membawa nanas dari Malaysia ke Zhengzhou, China, di Sepang hari ini.'] 

['China telah menjadi pasaran antarabangsa baru Malaysia untuk pelbagai nanas premium, MD2, berikutan kejayaan durian dalam menembusi negara itu pada Mei lepas.', '[MASK]', '"Industri nanas negara dijangka mempunyai kadar pertumbuhan tahunan sebanyak lima peratus setahun berdasarkan trend yang semakin meningkat dan penerimaan terhadap nanas Malaysia.